In [495]:
import pandas as pd
from pathlib import Path

In [496]:
from __future__ import print_function
import argparse
import os
import shutil
import random
import pandas as pd
from pathlib import Path
import csv


import ast
import numpy as np
import astor
import nltk

## Normal dataset

In [497]:
class RewriteName(ast.NodeTransformer):
#     def visit_Name(self, node, *args, **kwargs):
#         node.id = '@'
#         return node

    placeholder = '@'
    def visit_Assign(self, node):
        try:
            target = node.targets[0]
            if isinstance(target, ast.Name): 
                target.id = RewriteName.placeholder
            elif isinstance(target, ast.Subscript):
                value = target.slice.value
                if isinstance(value, ast.Str):
                    target.slice.value.s = RewriteName.placeholder
            else:
                return node
            targets = node.targets
            targets[0] = target
            return ast.Assign(targets=targets, value=node.value)
        except Exception as e:
            print(e)
            return node

def obfuscate_assignment(line):
    t = ast.parse(line)
    tree = RewriteName().visit(t)
    return astor.code_gen.to_source(tree)

In [498]:

# tokenizer = nltk.tokenize.RegexpTokenizer('[\s+\(\)]', gaps=True, discard_empty=True)
tokenizer = nltk.tokenize.RegexpTokenizer(r'[\w\./\']+|[^\w\s\']+', gaps=False, discard_empty=True)
# tokenizer = nltk.tokenize.WordPunctTokenizer()
for s in ['train2016 = add_date_features(train2016)', "df_all['product_title'] = 10", "@ = pd.read_csv('../input/sampleSubmission.csv')"]:
    print(tokenizer.tokenize(s))

['train2016', '=', 'add_date_features', '(', 'train2016', ')']
['df_all', '[', "'product_title'", ']', '=', '10']
['@', '=', 'pd.read_csv', '(', "'../input/sampleSubmission.csv'", ')']


In [502]:

def generate_dataset(root, name):
    path = os.path.join(root, name)
    if not os.path.exists(path):
        os.mkdir(path)

    p = '../epam_nlp_hack/results/py'
    sources = []
    targets = []
    for fn in sorted(list(Path(p).glob('*'))):
         with open(str(fn), encoding='utf8') as f:
            lines = f.read().replace('\\\n', '').split('\n')
            for line in lines:
                if (type(line) == str):
                    try:
                        parsed = ast.parse(line).body[0]
                        if isinstance(parsed, ast.Assign):
                            
                            sources.append(' '.join(tokenizer.tokenize(line)))
                            targets.append(' '.join(tokenizer.tokenize(obfuscate_assignment(line)[:-1])))
                    except Exception as e:
                        print(e)

            
#     p = '../epam_nlp_hack/minified/'

#     for fn in sorted(list(Path(p).glob('*'))):
#          with open(str(fn), encoding='utf8') as f:
#             text = f.read().replace('\\\n', '').split('\n')[:-1]
#             targets.extend(text)
            
#     filtered_sources = []
#     for line in sources:
#         if (type(line) == str):
#             try:
#                 parsed = ast.parse(line).body[0]
#                 if isinstance(parsed, ast.Assign):
#                     filtered_sources.append(line)
#             except:
#                 pass
    
#     filtered_targets = []
#     for line in targets:
#         if (type(line) == str):
#             try:
#                 parsed = ast.parse(line).body[0]
#                 if isinstance(parsed, ast.Assign):
#                     filtered_targets.append(line)
#             except:
#                 pass

    
    df = pd.DataFrame([sources, targets]).T.rename(columns={0: 'src', 1: 'tgt'})
    if name == 'train':
        df = df[:-500]
    if name == 'dev':
        df = df[-500:]
    data_path = os.path.join(path, 'data.txt')
    df.to_csv(data_path, sep='\t', header=None, line_terminator='\n', quoting=csv.QUOTE_ALL, index=False)
    # generate vocabulary
    vocab = set(' '.join(sources + minified).split())
    src_vocab = os.path.join(path, 'vocab.source')
    with open(src_vocab, 'w', encoding='utf8') as fout:
        fout.write("\n".join([str(i) for i in vocab]))
    tgt_vocab = os.path.join(path, 'vocab.target')
    shutil.copy(src_vocab, tgt_vocab)

In [503]:

data_dir = 'data'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

py_dir = os.path.join(data_dir, 'py')
if not os.path.exists(py_dir):
    os.mkdir(py_dir)

generate_dataset(py_dir, 'train')
generate_dataset(py_dir, 'dev')
# generate_dataset(toy_dir, 'test', 1000)

list index out of range
list index out of range
list index out of range
unexpected EOF while parsing (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
list index out of range
list index out of range
list index out of range
list index out of range
'ExtSlice' object has no attribute 'value'
list index out of range
list index out of range
list index out of range
list index out of range
unexpected EOF while parsing (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
list index out of range
list index out of range
unexpected EOF while parsing (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unex

In [483]:
# ast.parse('train2017.iloc[10]').body[0]
# t = ast.parse('train2016 = add_date_features(train2016)').body[0]
# name = t.targets[0]
# obfuscate_assignment("df_all['product_title'] = 10")

In [484]:
obfuscate_assignment('train2016 = add_date_features(train2016)')

'@ = add_date_features(train2016)\n'

In [485]:
a = ast.parse("df_all['product_title'] = 10").body[0]
s = a.targets[0]

'product_title'

In [493]:
pd.read_csv(py_dir + '/train/data.txt', sep='\t', header=None)

,0,1
0,properties2016 = pd.read_csv ( '../input/prope...,@ = pd.read_csv ( '../input/properties_2016.cs...
1,properties2017 = pd.read_csv ( '../input/prope...,@ = pd.read_csv ( '../input/properties_2017.cs...
2,train2016 = pd.read_csv ( '../input/train_2016...,@ = pd.read_csv ( '../input/train_2016_v2.csv'...
3,train2017 = pd.read_csv ( '../input/train_2017...,"@ = pd.read_csv ( '../input/train_2017.csv' , ..."
4,train2016 = add_date_features ( train2016 ),@ = add_date_features ( train2016 )
5,train2017 = add_date_features ( train2017 ),@ = add_date_features ( train2017 )
6,sample_submission = pd.read_csv ( '../input/sa...,@ = pd.read_csv ( '../input/sample_submission....
7,"train2016 = pd.merge ( train2016 , properties2...","@ = pd.merge ( train2016 , properties2016 , ho..."
8,"train2017 = pd.merge ( train2017 , properties2...","@ = pd.merge ( train2017 , properties2017 , ho..."
9,"train2017.iloc [:, train2017.columns.str.start...","train2017.iloc [:, ( train2017.columns.str.sta..."


In [504]:
lines = pd.read_csv(py_dir + '/train/data.txt', sep='\t', header=None)[0][:100]

In [494]:
# t = ast.parse('a = 3')
# body = t.body[0]
# target = b.targets[0]
# target.id

## Very simple dataset

In [253]:
def generate_dataset(root, name, size):
    path = os.path.join(root, name)
    if not os.path.exists(path):
        os.mkdir(path)

    # generate data file
    data_path = os.path.join(path, 'data.txt')
    with open(data_path, 'w') as fout:
        for _ in range(size):
            s = random.choice(['train', 'test'])
            tgt = [s, '=', 'pd.read_csv', "('../input/{}.csv')".format(s)]
            src = ['@', '=', 'pd.read_csv', "('../input/{}.csv')".format(s)]
            fout.write("\t".join([" ".join(src), " ".join(tgt)]))
            fout.write('\n')

    # generate vocabulary
    src_vocab = os.path.join(path, 'vocab.source')
    with open(src_vocab, 'w') as fout:
        fout.write("\n".join([str(i) for i in range(10)]))
    tgt_vocab = os.path.join(path, 'vocab.target')
    shutil.copy(src_vocab, tgt_vocab)

In [254]:
data_dir = 'data'

toy_dir = os.path.join(data_dir, 'toy_py')
if not os.path.exists(toy_dir):
    os.mkdir(toy_dir)

generate_dataset(toy_dir, 'train', 10000)
generate_dataset(toy_dir, 'dev', 1000)
generate_dataset(toy_dir, 'test', 1000)

In [259]:
pd.read_csv('data/toy_py' + '/train/data.txt', sep='\t')

,@ = pd.read_csv ('../input/train.csv'),train = pd.read_csv ('../input/train.csv')
0,@ = pd.read_csv ('../input/test.csv'),test = pd.read_csv ('../input/test.csv')
1,@ = pd.read_csv ('../input/train.csv'),train = pd.read_csv ('../input/train.csv')
2,@ = pd.read_csv ('../input/train.csv'),train = pd.read_csv ('../input/train.csv')
3,@ = pd.read_csv ('../input/train.csv'),train = pd.read_csv ('../input/train.csv')
4,@ = pd.read_csv ('../input/train.csv'),train = pd.read_csv ('../input/train.csv')
5,@ = pd.read_csv ('../input/train.csv'),train = pd.read_csv ('../input/train.csv')
6,@ = pd.read_csv ('../input/test.csv'),test = pd.read_csv ('../input/test.csv')
7,@ = pd.read_csv ('../input/test.csv'),test = pd.read_csv ('../input/test.csv')
8,@ = pd.read_csv ('../input/test.csv'),test = pd.read_csv ('../input/test.csv')
9,@ = pd.read_csv ('../input/train.csv'),train = pd.read_csv ('../input/train.csv')
